In [40]:
import pandas as pd

# Load CSV files
group_df = pd.read_csv("data/group_repair_results.csv")
models_df = pd.read_csv("data/models.csv")

# Merge on model_name == name
merged_df = group_df.merge(models_df, left_on='model_name', right_on='name')

# Group by model_name to compute stats
agg_df = merged_df.groupby('model_name').agg(
    avg_repairs=('total_repairs', lambda x: round(x.mean(), 2)),
    std_repairs=('total_repairs', lambda x: round(x.std(), 2)),
    avg_over=('overlapping_repairs', lambda x: round(x.mean(), 2)),
    std_over=('overlapping_repairs', lambda x: round(x.std(), 2)),
    avg_conflicting=('conflicting_repairs', lambda x: round(x.mean(), 2)),
    std_conflicting=('conflicting_repairs', lambda x: round(x.std(), 2)),
    avg_shared_locations=('shared_locations', lambda x: round(x.mean(), 2)),
    std_shared_locations=('shared_locations', lambda x: round(x.std(), 2))
).reset_index()

# Extract metadata: model_name -> ID and size (for sorting only)
meta_info = models_df[['name', 'idsmodels', 'size']].rename(columns={'name': 'model_name', 'idsmodels': 'Id'})

# Merge stats with metadata
final_df = agg_df.merge(meta_info, on='model_name', how='left')

# Sort by size descending, then drop size and model_name
final_df = final_df.sort_values('size', ascending=False).drop(columns=['size', 'model_name'])

# Reorder columns to have ID first
cols = ['Id'] + [col for col in final_df.columns if col != 'Id']
final_df = final_df[cols]

# Export to LaTeX
latex_table = final_df.to_latex(index=False, float_format="%.2f", caption="Repair Statistics by Model", label="tab:repair_stats", column_format='l' + 'r' * (len(final_df.columns) - 1))

# Save to .tex file
with open("data/repair_statistics_table.tex", "w") as f:
    f.write(latex_table)

print("LaTeX table saved to 'repair_statistics_table.tex'")


LaTeX table saved to 'repair_statistics_table.tex'
